In [1]:
import pandas as pd
from bs4 import BeautifulSoup as bs
from selenium import webdriver

table Tag는 크롤링하면 복잡하므로, pandas 라이브러리의 read_html(  ) 을 사용하면 된다.

In [2]:
# 접속 주소
url = 'https://comp.wisereport.co.kr/company/c1010001.aspx?cmp_cd='
# 종목의 코드 (SK하이닉스)
input_code = '000660'

### 연습
- webdriver의 Chrome을 이용하여 url + input_code 로 요청
- driver의 html 문서를 불러온다.
- 불러온 html을 BeautifulSoup을 이용하여 데이터 파싱
- div 태그 중 class의 이름이 'cmp_comment'와 같은 태그를 선택
    - li 태그들을 모두 찾아 텍스트를 추출해 새로운 리스트 생성 (cmp_info)
- div 태그 중 class의 이름이 'cmp_comment_etc'와 같은 태그를 선택
    - li 태그들을 모두 찾아 텍스트를 추출해 새로운 리스트 생성 (cmp_etc)
- DataFrame을 생성해 cmp_info, cmp_etc 대입

In [ ]:
driver = webdriver.Chrome()

In [9]:
driver.get( url + input_code )

In [ ]:
soup = bs( driver.page_source, 'html.parser' )
# 종료해도 됨

In [11]:
driver.quit()

In [12]:
# div 태그 중 class가 'cmp_comment'인 태그 선택
div_data = soup.find('div', attrs={'class' : 'cmp_comment'})
div_data

<div class="cmp_comment">
<ul class="dot_cmp">
<li class="dot_cmp" data-cd="000660">동사는 1949년 설립되어 경기도 이천시에 본사를 두고 4개의 생산기지와 3개의 연구개발법인 및 여러 해외 판매법인을 운영하는 글로벌 반도체 기업임.</li>
<li class="dot_cmp" data-cd="000660">메모리 반도체와 Foundry 사업을 영위하며, DRAM과 NAND Flash를 주력 생산하고 있음.</li>
<li class="dot_cmp" data-cd="000660">생성형 AI 시장의 신기술 등장과 시장 영향력 확대에 따라 고성능/고용량 메모리 솔루션 개발에 주력하고 있음.</li>
</ul>
</div>

In [14]:
# div_data에서 li 태그들을 모두 찾는다.
li_list = div_data.find_all('li')

In [ ]:
cmp_info = []
for li_data in li_list:
    # li_data는 Tag data 대입
    cmp_info.append( li_data.get_text() )

cmp_info

['동사는 1949년 설립되어 경기도 이천시에 본사를 두고 4개의 생산기지와 3개의 연구개발법인 및 여러 해외 판매법인을 운영하는 글로벌 반도체 기업임.',
 '메모리 반도체와 Foundry 사업을 영위하며, DRAM과 NAND Flash를 주력 생산하고 있음.',
 '생성형 AI 시장의 신기술 등장과 시장 영향력 확대에 따라 고성능/고용량 메모리 솔루션 개발에 주력하고 있음.']

In [18]:
# 다른 방법
cmp_etc = \
list(
    map(
        lambda x: x.get_text(),
        soup.find(
            'div', attrs={ 'class': 'cmp_comment_etc' }
        ).find_all('li')
    )
)
# 그대로 map()을 이용해 텍스트만 추출한 뒤 리스트에 집어넣을 수 있다.

cmp_etc

['2025년 상반기 전년동기 대비 연결기준 매출액은 38.2% 증가, 영업이익은 99.3% 증가, 당기순이익은 150.1% 증가.',
 'AI향 메모리 수요 성장과 우호적 가격 환경으로 DRAM과 NAND 출하량이 증가했고, 고부가가치 제품 판매로 실적이 개선됨.',
 'AI 서버와 데이터센터 확산으로 HBM 수요 급증, DRAM은 프리미엄 제품 성장이 예상되며 SSD 수요 증가로 NAND 수급 안정화 전망임.']

In [24]:
# 데이터프레임으로 뽑아낼 수도 있다.
df1 = \
pd.DataFrame(
    [
        cmp_info, cmp_etc
    ], index= ['cmp_info', 'cmp_etc']
)
df1

,0,1,2
cmp_info,동사는 1949년 설립되어 경기도 이천시에 본사를 두고 4개의 생산기지와 3개의 연...,"메모리 반도체와 Foundry 사업을 영위하며, DRAM과 NAND Flash를 주...",생성형 AI 시장의 신기술 등장과 시장 영향력 확대에 따라 고성능/고용량 메모리 솔...
cmp_etc,"2025년 상반기 전년동기 대비 연결기준 매출액은 38.2% 증가, 영업이익은 99...",AI향 메모리 수요 성장과 우호적 가격 환경으로 DRAM과 NAND 출하량이 증가했...,"AI 서버와 데이터센터 확산으로 HBM 수요 급증, DRAM은 프리미엄 제품 성장이..."


In [21]:
# 2개의 컬럼을 가지게 할 수도 있다.
df2 = \
pd.DataFrame(
    {
        'cmp_info': cmp_info,
        'cmp_etc': cmp_etc
    }
)

df2

,cmp_info,cmp_etc
0,동사는 1949년 설립되어 경기도 이천시에 본사를 두고 4개의 생산기지와 3개의 연...,"2025년 상반기 전년동기 대비 연결기준 매출액은 38.2% 증가, 영업이익은 99..."
1,"메모리 반도체와 Foundry 사업을 영위하며, DRAM과 NAND Flash를 주...",AI향 메모리 수요 성장과 우호적 가격 환경으로 DRAM과 NAND 출하량이 증가했...
2,생성형 AI 시장의 신기술 등장과 시장 영향력 확대에 따라 고성능/고용량 메모리 솔...,"AI 서버와 데이터센터 확산으로 HBM 수요 급증, DRAM은 프리미엄 제품 성장이..."


In [25]:
# df1의 인덱스를 컬럼으로, 컬럼을 인덱스로 변환하는 전치도 가능하다.
# 종목별로 모으기에는 df2가 더 직관적이다. 여러 종목을 모으는 경우를 위해 종목명 컬럼을 추가한다.
df2['code'] = input_code
df2

,cmp_info,cmp_etc,code
0,동사는 1949년 설립되어 경기도 이천시에 본사를 두고 4개의 생산기지와 3개의 연...,"2025년 상반기 전년동기 대비 연결기준 매출액은 38.2% 증가, 영업이익은 99...",000660
1,"메모리 반도체와 Foundry 사업을 영위하며, DRAM과 NAND Flash를 주...",AI향 메모리 수요 성장과 우호적 가격 환경으로 DRAM과 NAND 출하량이 증가했...,000660
2,생성형 AI 시장의 신기술 등장과 시장 영향력 확대에 따라 고성능/고용량 메모리 솔...,"AI 서버와 데이터센터 확산으로 HBM 수요 급증, DRAM은 프리미엄 제품 성장이...",000660


In [26]:
# 데이터프레임의 행 결합
    # 조건: 데이터프레임의 구조가 같다.
# pandas 안에 존재하는 concat() 함수의 결합
# concat() - 단순한 데이터프레임의 결합 (행으로 결합할 때 많이 쓰임)
    # 첫 번째 인자: 데이터프레임들 [df1, df2, ...] - 여러 개를 한 번에 결합하는 것도 가능
    # 두 번째 인자: axis 매개변수 -> 행으로 결합할지 열로 결합할지 결정
    # 세 번째 인자: ingore_index -> 결합 후 인덱스를 초기화할지 결정

import requests

In [27]:
input_code = '005180'

In [28]:
res = requests.get(url+input_code)
res

<Response [200]>

In [29]:
soup = bs(res.text, 'html.parser')

In [30]:
cmp_info = list(
    map(
        lambda x: x.get_text(),
        soup.find(
            'div', attrs={'class':'cmp_comment'}
        ).find_all('li')
    )
)

cmp_etc = list(
    map(
        lambda x: x.get_text(),
        soup.find(
            'div', attrs={'class':'cmp_comment_etc'}
        ).find_all('li')
    )
)

In [33]:
df3 = pd.DataFrame(
    {
        'cmp_info': cmp_info,
        'cmp_etc': cmp_etc
    }
)

df3['code'] = input_code
df3

,cmp_info,cmp_etc,code
0,동사는 1967년 설립된 유가증권시장 상장 유가공 전문기업임.,"2025년 상반기 전년동기 대비 연결기준 매출액은 1.4% 증가, 영업이익은 38....",005180
1,"바나나맛우유, 요플레, 투게더, 아카페라, 따옴 등 주요 제품을 보유하고 있으며, ...","유가공 사업은 원재료 가격 상승과 일회성 비용으로 수익성이 하락했으나, 고품질 정책...",005180
2,"남양주, 광주, 김해, 논산, 경산에 생산공장을 보유하고 있으며, 식품연구소 내 6...","국내는 기능성 제품 수요가 확산되고, 해외는 중국의 고품질 선호도, 베트남의 한류 ...",005180


In [ ]:
# df2, df3 결합
pd.concat( [df2, df3], axis=0 )

,cmp_info,cmp_etc,code
0,동사는 1949년 설립되어 경기도 이천시에 본사를 두고 4개의 생산기지와 3개의 연...,"2025년 상반기 전년동기 대비 연결기준 매출액은 38.2% 증가, 영업이익은 99...",000660
1,"메모리 반도체와 Foundry 사업을 영위하며, DRAM과 NAND Flash를 주...",AI향 메모리 수요 성장과 우호적 가격 환경으로 DRAM과 NAND 출하량이 증가했...,000660
2,생성형 AI 시장의 신기술 등장과 시장 영향력 확대에 따라 고성능/고용량 메모리 솔...,"AI 서버와 데이터센터 확산으로 HBM 수요 급증, DRAM은 프리미엄 제품 성장이...",000660
0,동사는 1967년 설립된 유가증권시장 상장 유가공 전문기업임.,"2025년 상반기 전년동기 대비 연결기준 매출액은 1.4% 증가, 영업이익은 38....",005180
1,"바나나맛우유, 요플레, 투게더, 아카페라, 따옴 등 주요 제품을 보유하고 있으며, ...","유가공 사업은 원재료 가격 상승과 일회성 비용으로 수익성이 하락했으나, 고품질 정책...",005180
2,"남양주, 광주, 김해, 논산, 경산에 생산공장을 보유하고 있으며, 식품연구소 내 6...","국내는 기능성 제품 수요가 확산되고, 해외는 중국의 고품질 선호도, 베트남의 한류 ...",005180
